In [1]:
from mavis import *

In [2]:
# 6.2
# Chiari input e outputs
theta = sp.symbols('theta')
sigma_1 = sp.symbols('sigma^2_1')
sigma_2 = sp.symbols('sigma^2_2')
sigma_X = sp.symbols('sigma^2_X')
sigma_Y = sp.symbols('sigma^2_Y')
sigma_XY = sp.symbols('sigma^2_XY')
eq1 = sp.Eq(theta, sp.S(1)/sp.S(2) * sp.atan( 2*sigma_XY / ( sigma_X-sigma_Y )) )
eq2 = sp.Eq(sigma_1, sp.S(1)/sp.S(2) * ( 2*sigma_XY / sp.sin(2*theta) + sigma_X + sigma_Y ) )
eq3 = sp.Eq(sigma_2, sigma_X+sigma_Y-sigma_1 )

display(eq1)
display(eq2)
display(eq3)

matCovTT = sp.MatrixSymbol('C_TT', 2, 2)
matCovTT_rhs = sp.Matrix([[sigma_X, sigma_XY],[sigma_XY, sigma_Y]])
ee = sp.Eq(matCovTT, matCovTT_rhs)
display(ee)

        ⎛   2⋅σ_XY__2   ⎞
    atan⎜───────────────⎟
        ⎝σ_X__2 - σ_Y__2⎠
θ = ─────────────────────
              2          

      σ_X__2   σ_XY__2    σ_Y__2
σ²₁ = ────── + ──────── + ──────
        2      sin(2⋅θ)     2   

σ²₂ = -σ²₁ + σ_X__2 + σ_Y__2

       ⎡σ_X__2   σ_XY__2⎤
C_TT = ⎢                ⎥
       ⎣σ_XY__2  σ_Y__2 ⎦

In [3]:
# 5.2
# C = C1 + C2 + C3
# C1 : vibrazioni, diagonale con varianze epsilon_tip e epsilon_tilt di MAVIS3, ricavate dalle 2 psd temporali.
# scegli una legge di controllo (funzioni di trasferimento noise e controllore(integratore) tutte in z) 
# e calcolare il residuo, vedere documento par. 4
# phi_N_tip e tilt sono varianza(del CoG, vedi sotto) /bias^2
# C2 = Caa - Cas Rt - R Cast + R Css Rt
# R = P_func(x,y)*w e' il ricostruttore dell 3 NGS, (vedi MAVIS_INT_REC)
# a e il vettore di tip tilt nella direzione di interesse (2) - Caa (2x2)
# s e il vettore di tip tilt nelle direzioni di NGS (6) - Css (6x6)
# calcolare Caa: Covarianza della direzione di interesse con se stessa, sia per tip che per tilt (2x2)
# calcolare Cas: Covarianza della direzione di interesse con le direzioni NGS (sono 3 o meno), sia per tip che per tilt (6x2)
# calcolare Css: Covarianza delle direzioni NGS (sono 3 o meno) fra loro, sia per tip che per tilt (6x6)
# C3 = R Cnn Rt
# R come sopra
# Cnn: vedi slides
# Cnn, matrice (6x6), diagonale, calcolo del sigma COG per le 3 direzioni, filtrato da un integratore semplice,

# C matrice di covarianza 2x2

matR = sp.MatrixSymbol('R', 2, 6)
matP = sp.MatrixSymbol('P_alpha', 2, 5)
matW = sp.MatrixSymbol('W', 5, 6)
display(sp.Eq(matR, sp.MatMul(matP,matW)) )

# calcolo del livello rumore (phi_noise) in funzione della 
# magnitudite della stella, strehl, FWHM, vedere documento mavis par.4 pag.24.:
# parte comune fra C1 e C3 
# C1 = vedi sezione 4 (diagonale) PSD temporale del tip e del tilt + rumore slopes (input 2 psd temporale tip/tilt + un livello di rumore)
matC1 = sp.MatrixSymbol('C_1', 2, 2)

matCaa = sp.MatrixSymbol('C_aa', 2, 2)
matCas = sp.MatrixSymbol('C_as', 2, 6)
matCss = sp.MatrixSymbol('C_ss', 6, 6)
matC2 = sp.MatrixSymbol('C_2', 2, 2)
display(sp.Eq(matC2,  matCaa - sp.MatMul(matCas, matR.T) - sp.MatMul(matR,matCas.T) + sp.MatMul(matR , matCss , matR.T)) )

matCnn = sp.MatrixSymbol('C_nn', 6, 6)
matC3 = sp.MatrixSymbol('C_3', 2, 2)
display(sp.Eq(matC3,  sp.MatMul(matR, matCnn, matR.T)) )

matC = sp.MatrixSymbol('C', 2, 2)
display(sp.Eq(matC,  matC1+matC2+matC3 ) )

R = Pₐₗₚₕₐ⋅W

           T       T          T      
C₂ = -Cₐₛ⋅R  -R⋅Cₐₛ  + R⋅Cₛₛ⋅R  + Cₐₐ

            T
C₃ = R⋅Cₙₙ⋅R 

C = C₁ + C₂ + C₃

In [4]:
P = interactionMatrixNGS()
display(P)
P = subsParamsByName(P, {'D':TelescopeDiameter, 'r_FoV':MCAO_FoV*arcsecsToRadians/2.0, 'H_DM':DM_height})
xx, yy = sp.symbols('x_1 y_1', real=True)
P = subsParamsByName(P, {'x_NGS':xx, 'y_NGS':yy})
P_func = sp.lambdify((xx, yy), P, modules='numpy')
print(cartTestNGSCoords0)
print(cartTestNGSCoords1)
print(cartTestNGSCoords2)
pp1 = P_func(cartTestNGSCoords[0][0]*arcsecsToRadians, cartTestNGSCoords[0][1]*arcsecsToRadians)
pp2 = P_func(cartTestNGSCoords[1][0]*arcsecsToRadians, cartTestNGSCoords[1][1]*arcsecsToRadians)
pp3 = P_func(cartTestNGSCoords[2][0]*arcsecsToRadians, cartTestNGSCoords[2][1]*arcsecsToRadians)
P_mat = np.vstack([pp1, pp2, pp3]) # aka Interaction Matrix, im
print(P_mat)

P_alpha1 = P_func(cartTestPointingCoords[0]*arcsecsToRadians, cartTestPointingCoords[1]*arcsecsToRadians)
# on axis version
P_alpha0 = P_func(0, 0)

rec_tomo = scipy.linalg.pinv2(P_mat) # aka W, 5x6

R_0 = np.dot(P_alpha0, rec_tomo)
R_1 = np.dot(P_alpha1, rec_tomo)

print(rec_tomo)
print(R_0)
print(R_1)

#pp1 = P_func(45*arcsecsToRadians,45*arcsecsToRadians)
#pp2 = P_func(-45*arcsecsToRadians,45*arcsecsToRadians)
#pp3 = P_func(0,-60*arcsecsToRadians)

⎡       4⋅√3⋅D⋅H_DM⋅x_NGS    2⋅√6⋅D⋅H_DM⋅y_NGS    2⋅√6⋅D⋅H_DM⋅x_NGS ⎤
⎢1  0  ───────────────────  ───────────────────  ───────────────────⎥
⎢                        2                    2                    2⎥
⎢      (D + 2⋅H_DM⋅r_FoV)   (D + 2⋅H_DM⋅r_FoV)   (D + 2⋅H_DM⋅r_FoV) ⎥
⎢                                                                   ⎥
⎢       4⋅√3⋅D⋅H_DM⋅y_NGS    2⋅√6⋅D⋅H_DM⋅x_NGS   -2⋅√6⋅D⋅H_DM⋅y_NGS ⎥
⎢0  1  ───────────────────  ───────────────────  ───────────────────⎥
⎢                        2                    2                    2⎥
⎣      (D + 2⋅H_DM⋅r_FoV)   (D + 2⋅H_DM⋅r_FoV)   (D + 2⋅H_DM⋅r_FoV) ⎦

[30.  0.]
[-8.68240888 49.24038765]
[-5.         -8.66025404]
[[ 1.          0.          0.42221252  0.          0.29854934]
 [ 0.          1.          0.          0.29854934 -0.        ]
 [ 1.          0.         -0.12219406  0.49002284 -0.08640425]
 [ 0.          1.          0.69299694 -0.08640425 -0.49002284]
 [ 1.          0.         -0.07036875 -0.08618377 -0.04975822]
 [ 0.          1.         -0.12188226 -0.04975822  0.08618377]]
[[ 0.1659003  -0.15351883  0.21901709  0.02296196  0.61508262  0.13055688]
 [-0.01174574  0.34082171 -0.1116391   0.09859392  0.12338484  0.56058436]
 [ 0.99933477  0.06355645 -0.14947145  0.60408168 -0.84986332 -0.66763813]
 [ 0.12251111  0.90023268  1.16442494 -0.13464866 -1.28693605 -0.76558402]
 [ 1.37527988  0.50750197 -0.57249372 -0.94365185 -0.80278616  0.43614989]]
[[ 0.1659003  -0.15351883  0.21901709  0.02296196  0.61508262  0.13055688]
 [-0.01174574  0.34082171 -0.1116391   0.09859392  0.12338484  0.56058436]]
[[ 0.31074966 -0.07900007  0.237

In [5]:
## The files in the folder are examples of data used or obtained in the jitter computation code. The parameters are the ones defined in params4jittercode.txt, unless otherwise specified. The files are the following:

#- example_gaussian_psf.fits: Gaussian PSF for the 2nd star, normalized so that a PSF with SR of 1 would have had a flux of 1 e-, i. e. it takes into account the loss of flux in the PSF core due to SR < 1 and FWHM > diffraction.
#- example_slope_noise.fits: Slope standard deviation (in mas) and bias factor in the case of the 2nd star sensed at 500 Hz.: due valori sigma e mu/shift

#- windpsd_mavis.fits: PSD of tip and tilt (in nm2/Hz) due to wind/vibrations for MAVIS. Format: [nfreq,3]. 1st index: frequency in Hz. 2nd index: Tip PSD. 3rd index: Tilt PSD.
#- minim_wind_noise12071nm2_psdmavis.fits: Table of residual variance on tip and tilt (in nm2) due to wind/vibrations for all the considered gains of the double integrator. Computed for the 2nd star sensed at 500 Hz. The noise level of 12071 nm2 corresponds to the slope error in example_slope_noise.fits. Format: [ngains,ngains,2].
#- psd_turb.fits: PSD of turbulent tip and tilt in nm2/Hz. Format: [nfreq,3]. 1st index: frequency in Hz. 2nd index: Tip PSD. 3rd index: Tilt PSD.
#- res_noise_ngs.fits: Residual variance of tip and tilt (in nm2) for all gains explored in the noise computation. Computed for the 2nd star at 500 Hz.

#- Cn.fits: Noise covariance matrix for all stars sensed at 500 Hz.

#- rec_tomo.fits: Tomographic reconstructor.

#- cov_tomo_onaxis.fits: Resulting covariance matrix of tip/tilt on axis for the tomography contribution. : C3
#- cov_noise_onaxis.fits: Resulting covariance matrix of tip/tilt on axis for the noise contribution. Computed using Cn.fits. : C3
# ok sarebbe le 9 C per le 9 direzioni specificate nei parametri
#- jitter_map.fits: Resulting jitter map for the configuration described in params4jittercode.txt. Format: [ndirections,5]. 1st & 2nd indexes: X, Y coordinates in arcsec. 3rd & 4th indexes: tip/tilt standard deviation along the major & minor axes (in mas). 5th index: Angle between the major axis and the X axis in degrees (counterclockwise).

filenames = ['Cn.fits', 'example_gaussian_psf.fits', 'psd_turb.fits',  'res_noise_ngs.fits', 'cov_noise_onaxis.fits',
             'cov_tomo_onaxis.fits', 'jitter_map.fits', 'rec_tomo.fits', 'example_slope_noise.fits']

# torna simile se moltiplico tutte le posizioni delle stelle per 2
filenames1 = ['rec_tomo.fits']

# torna simile se moltiplico tutte le posizioni delle stelle per 2
filenames2 = ['res_noise_ngs.fits']

# 'windpsd_maory.fits', 'minim_wind_noise12071nm2_psdmaory.fits',
# 'windpsd_mavis.fits', 'minim_wind_noise12071nm2_psdmavis.fits',
for name in filenames1:
    hdul = fits.open('MAVIS_TESTS/' + name)
    _data = np.asarray(hdul[0].data, np.float64)
    print(_data.shape)
#    if len(_data.shape) == 2:
#        plt.figure(figsize=(5, 5))
#        plt.subplot(1, 1, 1)
#        plt.imshow(_data, origin='lower', interpolation='nearest')
    #if len(_data.shape) == 1:
    print(_data)
#    plt.yscale('log')
#    plt.plot(_data[0]) 
#    plt.plot(_data[1]) 

(5, 6)
[[ 0.1659003  -0.15351883  0.21901708  0.02296196  0.61508262  0.13055688]
 [-0.01174574  0.34082171 -0.1116391   0.09859392  0.12338484  0.56058437]
 [ 0.99933462  0.06355644 -0.14947143  0.60408158 -0.84986319 -0.66763802]
 [ 0.12251109  0.90023251  1.16442474 -0.13464863 -1.28693583 -0.76558388]
 [ 1.37527965  0.50750188 -0.57249362 -0.94365169 -0.80278603  0.4361498 ]]


In [6]:
# programma call e domande
# rivedere parametri, fissare scelta per opzioni multiple, chiarire dove sono usati alcuni
# rivedere files di dati: chiarire cosa sono alcuni
# calcolo ricostruttore, fatto re 2 su coordinate refs, ecc ecc: OK
# come si assembla C1: diagonale, ottimizzazione per tip e tilt con i due guadagni, 
# si calcola usando una stella sola, la piu' brillante cioe' la seconda
# non viene usato R nel calcolo, rimane diagonale

# dettagli su MAVIS3: i conti non tornano....

# parametri calcolo gaussiana
# dati intermedi: richiedere valori calcolati di alcuni dati intermedi i.e.: mu e sigma della gaussiana, C, matrici varie
